In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
import re
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression

from catboost import CatBoostClassifier

In [2]:
df0 = pd.read_csv('marketing_dealer.csv', sep=';', error_bad_lines=False)

In [3]:
df0.head()

,id,name
0,1,Moi_vibor_WB
1,2,Akson
2,3,Bafus
3,5,Castorama
4,6,Cubatora


In [4]:
df1 = pd.read_csv('marketing_dealerprice.csv', sep=';')

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20416 entries, 0 to 20415
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            20416 non-null  int64  
 1   product_key   20416 non-null  object 
 2   price         20416 non-null  float64
 3   product_url   20182 non-null  object 
 4   product_name  20416 non-null  object 
 5   date          20416 non-null  object 
 6   dealer_id     20416 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 1.1+ MB


In [6]:
df1.head()

,id,product_key,price,product_url,product_name,date,dealer_id
0,2,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-11,2
1,3,546408,175.0,https://akson.ru//p/kontsentrat_prosept_multip...,"Концентрат Prosept Multipower для мытья полов,...",2023-07-11,2
2,4,546234,285.0,https://akson.ru//p/sredstvo_dlya_chistki_lyus...,Средство для чистки люстр Prosept Universal An...,2023-07-11,2
3,5,651258,362.0,https://akson.ru//p/udalitel_rzhavchiny_prosep...,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 0...",2023-07-11,2
4,6,546355,205.0,https://akson.ru//p/sredstvo_moyushchee_dlya_b...,Средство моющее для бани и сауны Prosept Multi...,2023-07-11,2


In [7]:
df2 = pd.read_csv('marketing_product.csv', sep=';')

In [8]:
df2.head(20)

,Unnamed: 0,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
0,0,245,008-1,4.680008e+12,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,360.00,858.0,20.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,189522705.0,150033482.0,008-1,NaN
1,1,3,242-12,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,NaN,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,NaN,NaN,NaN,NaN,NaN
2,2,443,0024-06 с,4.680008e+12,"Герметик акриловый цвет сосна, ф/п 600мл",307.00,644.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл",Герметик акриловый для швов для деревянных дом...,189522735.0,150126217.0,0024-06-с,NaN
3,3,147,305-2,4.610093e+12,Кондиционер для белья с ароматом королевского...,157.73,342.0,29.0,"Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...",339377922.0,150032962.0,305-2,NaN
4,4,502,0024-7 б,NaN,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN,NaN,189522867.0,150126216.0,0024-7-б,NaN
5,5,220,051-6,4.610093e+12,Грунт БЕТОНКОНТАКТготовый состав / 6 кг,703.00,1339.0,26.0,"Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...",453193200.0,149699633.0,051-6,NaN
6,6,385,051-10,4.680008e+12,Грунт БЕТОНКОНТАКТготовый состав / 12 кг,1111.00,2505.0,26.0,"Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...",189522783.0,149699627.0,051-10,NaN
7,7,114,125-5,4.680008e+12,"Средство для удаления технических масел, смазо...",711.82,1663.0,35.0,"Средство для удаления технических масел, смазо...","Средство для удаления технических масел, смазо...","Средство для удаления технических масел, смазо...",189522869.0,149992148.0,125-5,NaN
8,8,505,СФ001-5,4.610093e+12,Антисептик универсальный суперсильный,200.00,510.0,20.0,Антисептик универсальный суперсильный,Антисептик универсальный суперсильный,Антисептик универсальный суперсильный,NaN,NaN,NaN,NaN
9,9,32,100-1,4.680008e+12,Средство для мытья светлых полов с отбеливающи...,132.72,311.0,40.0,Профессиональное средство для мытья светлых по...,Профессиональное средство для мытья светлых по...,Профессиональное средство для мытья светлых по...,451088021.0,149705841.0,100-1,NaN


In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         496 non-null    int64  
 1   id                 496 non-null    int64  
 2   article            496 non-null    object 
 3   ean_13             464 non-null    float64
 4   name               494 non-null    object 
 5   cost               491 non-null    float64
 6   recommended_price  491 non-null    float64
 7   category_id        447 non-null    float64
 8   ozon_name          458 non-null    object 
 9   name_1c            485 non-null    object 
 10  wb_name            455 non-null    object 
 11  ozon_article       365 non-null    float64
 12  wb_article         340 non-null    float64
 13  ym_article         337 non-null    object 
 14  wb_article_td      32 non-null     object 
dtypes: float64(6), int64(2), object(7)
memory usage: 58.2+ KB


In [10]:
df3 = pd.read_csv('marketing_productdealerkey.csv', sep=';')
df3.head()

,id,key,dealer_id,product_id
0,1,546227,2,12
1,2,651265,2,106
2,3,546257,2,200
3,4,546408,2,38
4,5,651258,2,403


In [11]:
df2_copy = df2.copy(deep=True)

In [12]:
df2_copy = df2_copy[['id', 'name', 'cost', 'recommended_price', 'ozon_name', 'name_1c', 'wb_name']]
df2_copy.head()

,id,name,cost,recommended_price,ozon_name,name_1c,wb_name
0,245,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,360.00,858.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...
1,3,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,NaN
2,443,"Герметик акриловый цвет сосна, ф/п 600мл",307.00,644.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл",Герметик акриловый для швов для деревянных дом...
3,147,Кондиционер для белья с ароматом королевского...,157.73,342.0,"Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose..."
4,502,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN


## Работа с базой данных товаров Просепта

In [13]:
df2_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 496 non-null    int64  
 1   name               494 non-null    object 
 2   cost               491 non-null    float64
 3   recommended_price  491 non-null    float64
 4   ozon_name          458 non-null    object 
 5   name_1c            485 non-null    object 
 6   wb_name            455 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 27.2+ KB


In [14]:
is_null = df2_copy.isnull()

In [15]:
is_null.head()

,id,name,cost,recommended_price,ozon_name,name_1c,wb_name
0,False,False,False,False,False,False,False
1,False,False,False,False,True,False,True
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,True,True,True,True,True


In [16]:
row_with_null = is_null.any(axis=1)
rows_with_null = df2_copy[row_with_null]

In [17]:
rows_with_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41 entries, 1 to 430
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 41 non-null     int64  
 1   name               39 non-null     object 
 2   cost               36 non-null     float64
 3   recommended_price  36 non-null     float64
 4   ozon_name          3 non-null      object 
 5   name_1c            30 non-null     object 
 6   wb_name            0 non-null      object 
dtypes: float64(2), int64(1), object(4)
memory usage: 2.6+ KB


In [18]:
rows_with_null

,id,name,cost,recommended_price,ozon_name,name_1c,wb_name
1,3,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.00,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,NaN
4,502,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN
15,43,"Средство для мытья полов и стенMultipower ""Пол...",77.94,182.00,NaN,"Средство для мытья полов и стенMultipower ""Пол...",NaN
23,503,NaN,NaN,NaN,NaN,NaN,NaN
24,44,"Средство для мытья полов и стен Multipower ""По...",77.94,182.00,NaN,"Средство для мытья полов и стен Multipower ""По...",NaN
31,118,Средство усиленного действия для чистки коптил...,3730.00,8712.00,NaN,NaN,NaN
32,119,Средство низкопенное для очистки алюминия и...,2818.00,6581.00,NaN,NaN,NaN
35,504,NaN,NaN,NaN,NaN,NaN,NaN
46,182,Cредство для рук гигиеническое с антибактериал...,565.00,1319.00,NaN,NaN,NaN
47,181,Cредство для рук гигиеническое с антибактериал...,155.00,361.00,NaN,NaN,NaN


In [19]:
# заменим значения nan в столбцах ozon_name, name_1c, wb_name на значения name
name_list = ['ozon_name', 'name_1c', 'wb_name']
for i in name_list:
    df2_copy[i] = df2_copy[i].fillna(df2_copy['name'])

In [20]:
df2_copy.head(10)

,id,name,cost,recommended_price,ozon_name,name_1c,wb_name
0,245,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,360.00,858.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...
1,3,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,Антигололед - 32 PROSEPTготовый состав / 12 кг,Антигололед - 32 PROSEPTготовый состав / 12 кг,Антигололед - 32 PROSEPTготовый состав / 12 кг
2,443,"Герметик акриловый цвет сосна, ф/п 600мл",307.00,644.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл",Герметик акриловый для швов для деревянных дом...
3,147,Кондиционер для белья с ароматом королевского...,157.73,342.0,"Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose..."
4,502,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,"Герметик акриловой цвет Белый, 7 кг","Герметик акриловой цвет Белый, 7 кг","Герметик акриловой цвет Белый, 7 кг"
5,220,Грунт БЕТОНКОНТАКТготовый состав / 6 кг,703.00,1339.0,"Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ..."
6,385,Грунт БЕТОНКОНТАКТготовый состав / 12 кг,1111.00,2505.0,"Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ..."
7,114,"Средство для удаления технических масел, смазо...",711.82,1663.0,"Средство для удаления технических масел, смазо...","Средство для удаления технических масел, смазо...","Средство для удаления технических масел, смазо..."
8,505,Антисептик универсальный суперсильный,200.00,510.0,Антисептик универсальный суперсильный,Антисептик универсальный суперсильный,Антисептик универсальный суперсильный
9,32,Средство для мытья светлых полов с отбеливающи...,132.72,311.0,Профессиональное средство для мытья светлых по...,Профессиональное средство для мытья светлых по...,Профессиональное средство для мытья светлых по...


In [21]:
#заменим пропуски в cost ценами на тот же товар другого цвета
#cost_nan = df2_copy[df2_copy['id'] > 449]

In [22]:
#cost_nan.head(20)

In [23]:
cost_list = ['cost','recommended_price']
for i in cost_list:
    df2_copy[i] = df2_copy[i].fillna(0)
df2_copy.head()

,id,name,cost,recommended_price,ozon_name,name_1c,wb_name
0,245,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,360.00,858.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...
1,3,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,Антигололед - 32 PROSEPTготовый состав / 12 кг,Антигололед - 32 PROSEPTготовый состав / 12 кг,Антигололед - 32 PROSEPTготовый состав / 12 кг
2,443,"Герметик акриловый цвет сосна, ф/п 600мл",307.00,644.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл",Герметик акриловый для швов для деревянных дом...
3,147,Кондиционер для белья с ароматом королевского...,157.73,342.0,"Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose..."
4,502,"Герметик акриловой цвет Белый, 7 кг",0.00,0.0,"Герметик акриловой цвет Белый, 7 кг","Герметик акриловой цвет Белый, 7 кг","Герметик акриловой цвет Белый, 7 кг"


In [24]:
df2_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 496 non-null    int64  
 1   name               494 non-null    object 
 2   cost               496 non-null    float64
 3   recommended_price  496 non-null    float64
 4   ozon_name          494 non-null    object 
 5   name_1c            494 non-null    object 
 6   wb_name            494 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 27.2+ KB


In [25]:
df2_copy = df2_copy.dropna()
df2_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 494 entries, 0 to 495
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 494 non-null    int64  
 1   name               494 non-null    object 
 2   cost               494 non-null    float64
 3   recommended_price  494 non-null    float64
 4   ozon_name          494 non-null    object 
 5   name_1c            494 non-null    object 
 6   wb_name            494 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 30.9+ KB


In [26]:
# поиск явных дубликатов
print('явных дубликатов', df2_copy.duplicated().sum())

явных дубликатов 0


In [27]:
df2_copy_base = df2_copy[['id', 'name']]
df2_copy_base.head()

,id,name
0,245,Антисептик невымываемыйPROSEPT ULTRAконцентрат...
1,3,Антигололед - 32 PROSEPTготовый состав / 12 кг
2,443,"Герметик акриловый цвет сосна, ф/п 600мл"
3,147,Кондиционер для белья с ароматом королевского...
4,502,"Герметик акриловой цвет Белый, 7 кг"


## Работа с базой данных дилеров

In [28]:
df1_copy = df1[['product_key', 'product_name']]

In [29]:
df1_copy.head()

,product_key,product_name
0,546227,Средство универсальное Prosept Universal Spray...
1,546408,"Концентрат Prosept Multipower для мытья полов,..."
2,546234,Средство для чистки люстр Prosept Universal An...
3,651258,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 0..."
4,546355,Средство моющее для бани и сауны Prosept Multi...


In [30]:
df3_copy = df3[['key', 'product_id']]

In [31]:
df3_copy.rename(columns={'key':'product_key'}, inplace=True)

C:\Users\1\practicum\practicum_env\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [32]:
df3_copy.head()

,product_key,product_id
0,546227,12
1,651265,106
2,546257,200
3,546408,38
4,651258,403


In [33]:
#df1_copy.merge(df3_copy['product_id'], on='product_key', how='left')

In [34]:
df1_copy_base = pd.merge(df1_copy, df3_copy, how='left', on='product_key')

In [35]:
df1_copy_base.head()

,product_key,product_name,product_id
0,546227,Средство универсальное Prosept Universal Spray...,12.0
1,546408,"Концентрат Prosept Multipower для мытья полов,...",38.0
2,546234,Средство для чистки люстр Prosept Universal An...,18.0
3,651258,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 0...",403.0
4,546355,Средство моющее для бани и сауны Prosept Multi...,39.0


In [36]:
df1_copy_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20416 entries, 0 to 20415
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_key   20416 non-null  object 
 1   product_name  20416 non-null  object 
 2   product_id    17567 non-null  float64
dtypes: float64(1), object(2)
memory usage: 638.0+ KB


In [37]:
#df1_copy_base = df1_copy_base[['product_name', 'product_id']]

In [38]:
#df1_copy_base.head()

In [39]:
df1_copy_base.rename(columns={'product_id':'id'}, inplace=True)
#df1_copy_base['id'] = df1_copy_base['id'].astype('int')
#df1_copy_base.info()

In [40]:
is_null_1 = df1_copy_base.isnull()
row_with_null_1 = is_null_1.any(axis=1)
rows_with_null_1 = df1_copy_base[row_with_null_1]
rows_with_null_1.head(20)

,product_key,product_name,id
513,1001472231,"ОгнебиозащитадлядревесиныProseptОГНЕБИОPROFII,...",NaN
554,932001058,Удалитель мха Prosept концентрат 1 л,NaN
592,932001057,Удалитель мха Prosept готовый состав 5 л,NaN
593,716501,"Средство для устранения засоров в трубах 0,75л...",NaN
629,546241,Концентрат универсальный Prosept Bath Uni с ан...,NaN
632,382751,Огнебиозащита PROSEPT prof 2 группа для наружн...,NaN
651,716501,"Средство для устранения засоров в трубах 0,75л...",NaN
701,932001057,Удалитель мха Prosept готовый состав 5 л,NaN
702,932001058,Удалитель мха Prosept концентрат 1 л,NaN
705,https://kub02.ru/catalog/professionalnaya_byto...,Удалитель высолов PROSEPT SALT CLEANER (1л) со...,NaN


In [41]:
rows_with_null_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2849 entries, 513 to 20415
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_key   2849 non-null   object 
 1   product_name  2849 non-null   object 
 2   id            0 non-null      float64
dtypes: float64(1), object(2)
memory usage: 89.0+ KB


In [42]:
df1_copy_base = df1_copy_base.dropna()
df1_copy_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17567 entries, 0 to 20413
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_key   17567 non-null  object 
 1   product_name  17567 non-null  object 
 2   id            17567 non-null  float64
dtypes: float64(1), object(2)
memory usage: 549.0+ KB


In [43]:
df1_copy_base['id'] = df1_copy_base['id'].astype('int64')
#df1_copy_base['product_key'] = df1_copy_base['product_key'].astype('int64')
#df1_copy_bases = df1_copy_base[['product_name', 'id']]
df1_copy_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17567 entries, 0 to 20413
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_key   17567 non-null  object
 1   product_name  17567 non-null  object
 2   id            17567 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 549.0+ KB


In [44]:
df1_copy_base.head()

,product_key,product_name,id
0,546227,Средство универсальное Prosept Universal Spray...,12
1,546408,"Концентрат Prosept Multipower для мытья полов,...",38
2,546234,Средство для чистки люстр Prosept Universal An...,18
3,651258,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 0...",403
4,546355,Средство моющее для бани и сауны Prosept Multi...,39


In [45]:
# поиск явных дубликатов
print('явных дубликатов', df1_copy_base.duplicated().sum())

явных дубликатов 15876


## Bert

## Обработка строчных данных ##

In [46]:
def clear_text_jen(text):
    text = re.sub(r'"{2}|[/]', ' ', text) # убирает 2 подряд кавычки (") + убирает слэш
    text = re.sub(r'(?<=[а-я])[A-Z]', ' \g<0>', text) # разделяет пробелом слип. англ. и русское слова
    text = re.sub(r'[A-Za-z](?=[а-я])', '\g<0> ', text) # разделяет пробелом слип. англ. и русское слова
    text = re.sub(r'(?<=[А-Я]{2})[а-я]', ' \g<0>', text) # разделяет пробелом слип. русские слова
    text = re.sub(r'\W', ' ', text) # убирает знаки препинания
    return ' '.join(text.split())

In [47]:
corpus = list(df2_copy_base['name'])

def lemmatize_sent(text): 
    # Text input is string, returns lowercased strings.
    doc = [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) for word, tag in pos_tag(nltk.word_tokenize(text))]
    return " ".join(doc)


def lemmatize(text):
    m = Mystem()
    lemm_list = m.lemmatize(text.lower())
    lemm_text = "".join(lemm_list)
    return lemm_text


def clear_text(text):
    clean_text = re.sub(r'[^a-zA-Z_0-9а-яА-ЯёЁ]',' ',text.lower())
    clean_text = " ".join(clean_text.split())
    return clean_text

def clear_text_nltk(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('russian'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    filtered_tokens = " ".join(filtered_tokens)
    return filtered_tokens

def lemmatize_nltk(text):
    m = SnowballStemmer("russian")
    lemm_list = m.stem(text)
    lemm_text = "".join(lemm_list)
    return lemm_text


In [48]:
#df2_copy_base['name'] = df2_copy_base['name'].apply(clear_text_nltk)
#display(df2_copy_base.head())

In [49]:
df2_copy_base['name'] = df2_copy_base['name'].apply(clear_text_jen)
display(df2_copy_base.head())

C:\Users\1\AppData\Local\Temp/ipykernel_9324/3601381022.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_copy_base['name'] = df2_copy_base['name'].apply(clear_text_jen)


,id,name
0,245,Антисептик невымываемый PROSEPT ULTRA концентр...
1,3,Антигололед 32 PROSEPT готовый состав 12 кг
2,443,Герметик акриловый цвет сосна ф п 600мл
3,147,Кондиционер для белья с ароматом королевского ...
4,502,Герметик акриловой цвет Белый 7 кг


In [50]:
df2_copy_base['name'] = df2_copy_base['name'].apply(clear_text)
display(df2_copy_base.head())

C:\Users\1\AppData\Local\Temp/ipykernel_9324/3050941631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_copy_base['name'] = df2_copy_base['name'].apply(clear_text)


,id,name
0,245,антисептик невымываемый prosept ultra концентр...
1,3,антигололед 32 prosept готовый состав 12 кг
2,443,герметик акриловый цвет сосна ф п 600мл
3,147,кондиционер для белья с ароматом королевского ...
4,502,герметик акриловой цвет белый 7 кг


In [51]:
tqdm.pandas()
# обработаем данные

df2_copy_base['lemm_name'] = df2_copy_base['name'].progress_apply(lemmatize)
df2_copy_base = df2_copy_base.drop(['name'], axis=1)

100%|████████████████████████████████████████████████████████████████████████████████| 494/494 [10:30<00:00,  1.28s/it]
C:\Users\1\AppData\Local\Temp/ipykernel_9324/2256259791.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_copy_base['lemm_name'] = df2_copy_base['name'].progress_apply(lemmatize)


In [52]:
display(df2_copy_base.head(20))

,id,lemm_name
0,245,антисептик невымываемый prosept ultra концентр...
1,3,антигололед 32 prosept готовый состав 12 кг\n
2,443,герметик акриловый цвет сосна ф п 600мл\n
3,147,кондиционер для белье с аромат королевский ири...
4,502,герметик акриловый цвет белый 7 кг\n
5,220,грунт бетонконтакте готовый состав 6 кг\n
6,385,грунт бетонконтакте готовый состав 12 кг\n
7,114,средство для удаление технический масло смазоч...
8,505,антисептик универсальный суперсильный\n
9,32,средство для мытье светлый пол с отбеливающий ...


In [53]:
df1_copy_base['product_name'] = df1_copy_base['product_name'].apply(clear_text_jen)
display(df1_copy_base.head())

,product_key,product_name,id
0,546227,Средство универсальное Prosept Universal Spray...,12
1,546408,Концентрат Prosept Multipower для мытья полов ...,38
2,546234,Средство для чистки люстр Prosept Universal An...,18
3,651258,Удалитель ржавчины PROSEPT RUST REMOVER 0 5л 0...,403
4,546355,Средство моющее для бани и сауны Prosept Multi...,39


In [54]:
df1_copy_base['product_name'] = df1_copy_base['product_name'].apply(clear_text)
display(df1_copy_base.head())

,product_key,product_name,id
0,546227,средство универсальное prosept universal spray...,12
1,546408,концентрат prosept multipower для мытья полов ...,38
2,546234,средство для чистки люстр prosept universal an...,18
3,651258,удалитель ржавчины prosept rust remover 0 5л 0...,403
4,546355,средство моющее для бани и сауны prosept multi...,39


In [55]:
df1_copy_bases_100 = df1_copy_base.iloc[0:100]
df1_copy_bases_100.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_key   100 non-null    object
 1   product_name  100 non-null    object
 2   id            100 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 3.1+ KB


In [56]:
tqdm.pandas()
# обработаем данные

df1_copy_bases_100['lemm_name'] = df1_copy_bases_100['product_name'].progress_apply(lemmatize)
df1_copy_bases_100 = df1_copy_bases_100.drop(['product_name'], axis=1)
display(df1_copy_bases_100.head(20))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:11<00:00,  1.32s/it]
C:\Users\1\AppData\Local\Temp/ipykernel_9324/1372103437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_copy_bases_100['lemm_name'] = df1_copy_bases_100['product_name'].progress_apply(lemmatize)


,product_key,id,lemm_name
0,546227,12,средство универсальный prosept universal spray...
1,546408,38,концентрат prosept multipower для мытье пол ци...
2,546234,18,средство для чистка люстра prosept universal a...
3,651258,403,удалитель ржавчина prosept rust remover 0 5л 0...
4,546355,39,средство мыть для баня и сауна prosept multipo...
5,831859,396,пропитка prosept aquaisol для камень концентра...
6,546406,40,средство для мытье плитка и керамогранит prose...
7,831858,397,пропитка prosept aquaisol для камень концентра...
8,857015,236,шпаклевка выравнивать акриловый prosept plasti...
9,651265,106,соль для посудомоечный машина prosept splash 1...


In [57]:
df1_copy_bases_100_200 = df1_copy_base.iloc[101:201]
df1_copy_bases_100_200.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 101 to 200
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_key   100 non-null    object
 1   product_name  100 non-null    object
 2   id            100 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 3.1+ KB


In [58]:
tqdm.pandas()
# обработаем данные

df1_copy_bases_100_200['lemm_name'] = df1_copy_bases_100_200['product_name'].progress_apply(lemmatize)
df1_copy_bases_100_200 = df1_copy_bases_100_200.drop(['product_name'], axis=1)
display(df1_copy_bases_100_200.head())

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:14<00:00,  1.34s/it]
C:\Users\1\AppData\Local\Temp/ipykernel_9324/2013115602.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_copy_bases_100_200['lemm_name'] = df1_copy_bases_100_200['product_name'].progress_apply(lemmatize)


,product_key,id,lemm_name
101,200544012,181,просепт professional средство для рука гигиени...
102,100070803,372,просепт герметик акриловый паропроницаемый 15 ...
103,200394764,368,просепт герметик акриловый паропроницаемый 15 ...
104,200652975,376,просепт герметик акриловый паропроницаемый 15 ...
105,200394762,365,просепт герметик акриловый паропроницаемый 600...


## Подготовка данных для обучения моделей

In [59]:
stopwords = set(stopwords.words('russian'))

In [60]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [61]:
# расчет tfdif в каждой из выборок

matrix_features_train = count_tf_idf.fit_transform(df2_copy_base['lemm_name'])
matrix_features_test = count_tf_idf.transform(df1_copy_bases_100['lemm_name'])
matrix_bases_100_200 = count_tf_idf.transform(df1_copy_bases_100_200['lemm_name'])
target_bases_100_200 = df1_copy_bases_100_200['product_key']
target_train = df2_copy_base['id']
target_test = df1_copy_bases_100['product_key']

In [62]:
print(matrix_features_train.shape)
print(matrix_features_test.shape)
print(target_train.shape)
print(target_test.shape)
print(matrix_bases_100_200.shape)
print(target_bases_100_200.shape)

(494, 472)
(100, 472)
(494,)
(100,)
(100, 472)
(100,)


In [66]:
target_bases_100_200 = df1_copy_bases_100_200['product_key']
target_train = df2_copy_base['id']
target_test = df1_copy_bases_100['product_key']

In [63]:
df_features_train = pd.DataFrame(matrix_features_train.toarray(), columns=count_tf_idf.get_feature_names())
df_features_train.head()

,05,10,100,11,12,120,14,15,150,16,...,штука,щадить,экзотический,эконом,эпоксидный,эффект,яблокаконцентрированный,яблоко,японский,яхтный
0,0.0,0.369861,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.400696,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
df_features_test = pd.DataFrame(matrix_features_test.toarray(), columns=count_tf_idf.get_feature_names())
df_features_test.head()

,05,10,100,11,12,120,14,15,150,16,...,штука,щадить,экзотический,эконом,эпоксидный,эффект,яблокаконцентрированный,яблоко,японский,яхтный
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.435863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
df_bases_100_200 = pd.DataFrame(matrix_bases_100_200.toarray(), columns=count_tf_idf.get_feature_names())
df_bases_100_200.head()

,05,10,100,11,12,120,14,15,150,16,...,штука,щадить,экзотический,эконом,эпоксидный,эффект,яблокаконцентрированный,яблоко,японский,яхтный
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.326239,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.582543,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.555147,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.563891,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


## Обучение моделей

In [66]:
#features_val = count_tf_idf.transform(df_val['lemm_name'])
#target_val = df_val['id']
knn = KNeighborsClassifier(n_neighbors=20, weights='distance', metric='cosine')
knn.fit(df_features_train, target_train)

# Находим ближайшие N соседей для каждого запроса в валидационной выборке
neighbors_val = knn.kneighbors(df_features_test, n_neighbors=20, return_distance=False)


In [67]:
# создаем датафрейм для обучения модели
train_data = pd.DataFrame(columns=['query_id', 'base_id', 'target'])

for i in range(len(neighbors_val)):
    query_id = target_test.iloc[i]
    for j in neighbors_val[i]:
        base_id = target_train.iloc[j]
        target = 1 if base_id == df1_copy_bases_100['id'][i] else 0
        train_data = train_data.append({'query_id': query_id, 'base_id': base_id, 'target': target}, ignore_index=True)


In [78]:
df_neighbors_val = pd.DataFrame(neighbors_val)

In [79]:
df_neighbors_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       100 non-null    int64
 1   1       100 non-null    int64
 2   2       100 non-null    int64
 3   3       100 non-null    int64
 4   4       100 non-null    int64
 5   5       100 non-null    int64
 6   6       100 non-null    int64
 7   7       100 non-null    int64
 8   8       100 non-null    int64
 9   9       100 non-null    int64
 10  10      100 non-null    int64
 11  11      100 non-null    int64
 12  12      100 non-null    int64
 13  13      100 non-null    int64
 14  14      100 non-null    int64
 15  15      100 non-null    int64
 16  16      100 non-null    int64
 17  17      100 non-null    int64
 18  18      100 non-null    int64
 19  19      100 non-null    int64
dtypes: int64(20)
memory usage: 15.8 KB


In [80]:
df_neighbors_val.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,486,341,176,312,433,444,255,261,260,104,339,179,134,445,449,362,359,8,66,110
1,63,174,89,268,267,273,60,419,271,417,424,46,145,23,88,15,9,351,95,302
2,211,213,208,362,261,255,359,461,260,433,444,104,324,66,341,486,110,456,491,307
3,251,464,282,144,43,249,192,193,204,36,49,41,190,194,206,406,413,468,48,320
4,248,198,210,219,249,407,243,410,402,396,273,245,60,46,444,433,486,341,134,179


In [68]:
train_data.head(40)

,query_id,base_id,target
0,546227,12,1
1,546227,13,0
2,546227,15,0
3,546227,69,0
4,546227,4,0
5,546227,5,0
6,546227,259,0
7,546227,260,0
8,546227,261,0
9,546227,262,0


In [69]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   query_id  2000 non-null   object
 1   base_id   2000 non-null   object
 2   target    2000 non-null   object
dtypes: object(3)
memory usage: 47.0+ KB


In [89]:
# создаем датафрейм для обучения модели
#train_data = pd.DataFrame(columns=['request_id', 'object_id', 'target'])

#for i in range(len(df_features_test)):
#    request_id = i
#    distances, indices = knn.kneighbors([df_features_test.iloc[i]])
#    neighbor_ids = indices[0]
#    for neighbor_id in neighbor_ids:
#        target = 1 if neighbor_id == i else 0
#        train_data = train_data.append({'request_id': request_id, 'object_id': neighbor_id, 'target': target}, ignore_index=True)

In [90]:
#train_data.head(30)

In [91]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   query_id  2000 non-null   object
 1   base_id   2000 non-null   object
 2   target    2000 non-null   object
dtypes: object(3)
memory usage: 47.0+ KB


In [92]:
#train_data['object_id'].unique()

In [93]:
#train_data['target'].unique()

In [94]:
#train_data['request_id'].unique()

In [70]:
train_data_int = train_data.astype('int64')
train_data_int.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   query_id  2000 non-null   int64
 1   base_id   2000 non-null   int64
 2   target    2000 non-null   int64
dtypes: int64(3)
memory usage: 47.0 KB


In [71]:
train_data_int.head()

,query_id,base_id,target
0,546227,12,1
1,546227,13,0
2,546227,15,0
3,546227,69,0
4,546227,4,0


In [72]:
# обучаем модель классификации
X_train = train_data_int[['query_id', 'base_id']]
y_train = train_data_int['target']

#classifier = LogisticRegression()
#classifier.fit(X_train, y_train)

In [73]:
model_cat = CatBoostClassifier(iterations=50, random_state=11)
model_cat.fit(X_train, y_train)

Learning rate set to 0.216032
0:	learn: 0.5446437	total: 190ms	remaining: 9.32s
1:	learn: 0.4448026	total: 192ms	remaining: 4.6s
2:	learn: 0.3751076	total: 193ms	remaining: 3.02s
3:	learn: 0.3247481	total: 194ms	remaining: 2.23s
4:	learn: 0.2877559	total: 196ms	remaining: 1.76s
5:	learn: 0.2602502	total: 197ms	remaining: 1.45s
6:	learn: 0.2398783	total: 199ms	remaining: 1.22s
7:	learn: 0.2246794	total: 200ms	remaining: 1.05s
8:	learn: 0.2129069	total: 202ms	remaining: 919ms
9:	learn: 0.2043660	total: 203ms	remaining: 813ms
10:	learn: 0.1979633	total: 205ms	remaining: 725ms
11:	learn: 0.1933193	total: 206ms	remaining: 652ms
12:	learn: 0.1897728	total: 207ms	remaining: 590ms
13:	learn: 0.1869782	total: 209ms	remaining: 537ms
14:	learn: 0.1848545	total: 210ms	remaining: 490ms
15:	learn: 0.1832821	total: 211ms	remaining: 449ms
16:	learn: 0.1818264	total: 213ms	remaining: 413ms
17:	learn: 0.1803889	total: 214ms	remaining: 380ms
18:	learn: 0.1796850	total: 215ms	remaining: 351ms
19:	learn: 0

In [74]:
neighbors_test = knn.kneighbors(df_bases_100_200, n_neighbors=20, return_distance=False)

In [105]:
# Обучаем классификатор на получившейся выборке
#knn_classifier = KNeighborsClassifier(n_neighbors=20, weights='distance', metric='cosine')
#knn_classifier.fit(X_train, y_train)

# Предсказываем вероятности для валидационной выборки
probs = knn.predict_proba(df_bases_100_200)

In [107]:
df_probs = pd.DataFrame(probs)
df_probs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 494 entries, 0 to 493
dtypes: float64(494)
memory usage: 386.1 KB


In [108]:
df_probs.head()

,0,1,2,3,4,5,6,7,8,9,...,484,485,486,487,488,489,490,491,492,493
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038799,0.038799,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.047798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.023151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.023809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.035581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
# проверяем модель на валидационной выборке
#validation_requests = pd.DataFrame(...)  # валидационная выборка запросов
val_data = pd.DataFrame(columns=['query_id', 'base_id'])

for i in range(len(neighbors_test)):
    query_id = target_bases_100_200.iloc[i]
    for j in neighbors_test[i]:
        base_id = j
        val_data = val_data.append({'query_id': query_id, 'base_id': base_id}, ignore_index=True)


In [89]:
val_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   query_id  2000 non-null   object
 1   base_id   2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [90]:
val_data_int = val_data.astype('int64')
val_data_int.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   query_id  2000 non-null   int64
 1   base_id   2000 non-null   int64
dtypes: int64(2)
memory usage: 31.4 KB


In [91]:
val_data_int.head(40)

,query_id,base_id
0,200544012,45
1,200544012,44
2,200544012,113
3,200544012,167
4,200544012,440
5,200544012,281
6,200544012,279
7,200544012,38
8,200544012,29
9,200544012,86


In [83]:
# проверяем модель на валидационной выборке
#validation_requests = pd.DataFrame(...)  # валидационная выборка запросов

#val_data = pd.DataFrame(columns=['request_id', 'object_id'])

#for i in range(len(df_bases_100_200)):
#    request_id = i
#    distances, indices = knn.kneighbors([df_bases_100_200.iloc[i]])
#    neighbor_ids = indices[0]
#    for neighbor_id in neighbor_ids:
#        val_data = val_data.append({'request_id': request_id, 'object_id': neighbor_id}, ignore_index=True)


In [84]:
#val_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   request_id  2000 non-null   object
 1   object_id   2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [85]:
#val_data_int = val_data.astype('int64')
#val_data_int.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   request_id  2000 non-null   int64
 1   object_id   2000 non-null   int64
dtypes: int64(2)
memory usage: 31.4 KB


In [92]:
# предсказываем вероятности для валидационной выборки
X_val = val_data_int
val_data_int['probability'] = model_cat.predict_proba(X_val)[:, 1]

In [93]:
val_data_int.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   query_id     2000 non-null   int64  
 1   base_id      2000 non-null   int64  
 2   probability  2000 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 47.0 KB


In [94]:
val_data_int.head()

,query_id,base_id,probability
0,200544012,45,0.031766
1,200544012,44,0.036769
2,200544012,113,0.047659
3,200544012,167,0.051224
4,200544012,440,0.010359


In [95]:
top_objects = val_data_int.groupby('query_id').apply(lambda x: x.nlargest(5, 'probability'))

In [96]:
top_objects.head(15)

query_id  base_id  probability
query_id                                       
1462335   1523    1462335      234     0.105898
          1533    1462335      217     0.082549
          1534    1462335      209     0.082549
          1539    1462335      172     0.067993
          1526    1462335      302     0.062234
100067707 1307  100067707      200     0.113505
          1302  100067707      202     0.102542
          1308  100067707      205     0.093917
          1304  100067707      253     0.074470
          1303  100067707       10     0.066940
100067708 1405  100067708      196     0.113505
          1408  100067708      193     0.113505
          1403  100067708      207     0.093917
          1407  100067708      204     0.093917
          1402  100067708      218     0.091830

In [109]:
precision_at_5 = 0
for i in target_bases_100_200:
    if i in top_objects['base_id']:
        precision_at_5 += 1
precision_at_5 /= len(top_objects)
print("Precision@5:", precision_at_5)

Precision@5: 0.0


In [101]:
df2[df2['id'] == 200]

,Unnamed: 0,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
317,317,200,114-0,4.680008e+12,Средство для мытья стекол и зеркалOptic Crysta...,78.0,182.0,51.0,"Средство для мытья стекол, зеркал и пластика P...","Средство для мытья стекол, зеркал и пластика P...","Средство для мытья стекол, зеркал и пластика P...",448763582.0,149811060.0,114-0,114-00


In [102]:
df1[df1['product_key'] == '100067707']

,id,product_key,price,product_url,product_name,date,dealer_id
166,165,100067707,3105.0,https://www.bafus.ru/100067707/,Просепт 50 отбеливатель для древесины (10 л) ж...,2023-07-11,3
2158,2158,100067707,3105.0,https://www.bafus.ru/100067707/,Просепт 50 отбеливатель для древесины (10 л) ж...,2023-07-12,3
3798,3920,100067707,3105.0,https://www.bafus.ru/100067707/,Просепт 50 отбеливатель для древесины (10 л) ж...,2023-07-13,3
5368,5506,100067707,3105.0,https://www.bafus.ru/100067707/,Просепт 50 отбеливатель для древесины (10 л) ж...,2023-07-14,3
6972,7066,100067707,3105.0,https://www.bafus.ru/100067707/,Просепт 50 отбеливатель для древесины (10 л) ж...,2023-07-17,3
8561,8622,100067707,3105.0,https://www.bafus.ru/100067707/,Просепт 50 отбеливатель для древесины (10 л) ж...,2023-07-18,3
9970,10113,100067707,3105.0,https://www.bafus.ru/100067707/,Просепт 50 отбеливатель для древесины (10 л) ж...,2023-07-19,3
11289,11432,100067707,3105.0,https://www.bafus.ru/100067707/,Просепт 50 отбеливатель для древесины (10 л) ж...,2023-07-21,3
12661,12763,100067707,3105.0,https://www.bafus.ru/100067707/,Просепт 50 отбеливатель для древесины (10 л) ж...,2023-07-24,3
14134,14199,100067707,3105.0,https://www.bafus.ru/100067707/,Просепт 50 отбеливатель для древесины (10 л) ж...,2023-07-25,3
